# Enunciado

Tras estudiar las estrategias de tratamiento de valores numéricos no disponibles como se explica en el [notebook de preprocesado del dataset housing](https://github.com/avidaldo/python-libs-ml/blob/master/end2end/e2e04_preprocessing.ipynb) , propón una nueva solución para utilizando la clase [KNNImputer de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html).


´KNNImputer´ es una clase proporcionada por scikit-learn que se utiliza para imputar valores faltantes en un conjunto de datos utilizando el algoritmo de los k-vecinos más cercanos


## Resolución


### Importamos los datos y creamos un objeto DataFrame


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# creamos un objeto DataFrame a partir de un csv
housing = pd.read_csv("./data/housing.csv")
# housing es un objeto DataFrame
print(type(housing))
# housing tiene la estructura
print(housing.dtypes)

# definimos las columnas interesantes para que sea mas cómodo de visualizar
printRows = ["housing_median_age", "median_income", "median_house_value"]

<class 'pandas.core.frame.DataFrame'>
longitude             float64
latitude              float64
housing_median_age    float64
total_rooms           float64
total_bedrooms        float64
population            float64
households            float64
median_income         float64
median_house_value    float64
ocean_proximity        object
dtype: object


### Generación de conjuntos de entrenamiento

- `train_test_split` from `sklearn.model_selection`


In [2]:
# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
# Divide el DataFrame housing en conjuntos de entrenamiento y prueba
train_set, test_set = train_test_split(
    housing,  # DataFrame
    test_size=0.2,  # conjunto de prueba será el 20% del tamaño total del conjunto de datos original
    stratify=pd.cut(  # realiza un muestreo estratificado basado en los valores
        # selecciono los datos de la columna "median_income"
        housing["median_income"],
        # Los datos se agrupan en intervalos definidos por los límites [0.0, 1.5, 3.0, 4.5, 6.0, np.inf]
        bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
        # los datos ee etiquetan con valores [1, 2, 3, 4, 5]
        labels=[1, 2, 3, 4, 5],
    ),
)

#### Tengo definido dos conjuntos de datos: `train_set` y `test_set`.

- Conjunto de entrenamiento (train_set):

  - Este conjunto de datos se utiliza para entrenar el modelo de aprendizaje automático.
  - Los modelos se ajustan a los datos de entrenamiento para aprender relaciones entre las características (variables independientes) y la variable objetivo (variable dependiente).

- Conjunto de prueba (test_set):

  - Este conjunto de datos se utiliza para evaluar el rendimiento del modelo entrenado en datos no vistos.
  - Después de entrenar el modelo en el conjunto de entrenamiento, se utiliza el conjunto de prueba para estimar cómo se generalizará el modelo a nuevos datos.
  - Esto es importante para evaluar la capacidad del modelo para hacer predicciones precisas sobre datos que no ha visto durante el entrenamiento.


In [3]:
# Definimos las columnas interesantes para que sea más cómodo de visualizar
# printRows = ["housing_median_age", "median_income", "median_house_value"]

# Seguimos trabajando con objetos DataFrame
print("train_set type =", type(train_set))
print("test_set type =", type(test_set))

# Conjunto de entrenamiento (train_set):
print("[conjuntos de datos: train_set]\n", train_set[printRows].head(2))

# Conjunto de prueba (test_set):
print("[conjuntos de datos: test_set]\n", test_set[printRows].head(2))

train_set type = <class 'pandas.core.frame.DataFrame'>
test_set type = <class 'pandas.core.frame.DataFrame'>
[conjuntos de datos: train_set]
       housing_median_age  median_income  median_house_value
7525                37.0         1.7158             94300.0
6243                35.0         3.7454            156900.0
[conjuntos de datos: test_set]
        housing_median_age  median_income  median_house_value
15946                52.0         3.8274            252000.0
18974                22.0         1.6875             87500.0


#### Procesado de datos:

##### Eliminamos de `train_set` la columna que queremos usar como entrada del modelo de entrenamiento

- Se elimina la columna `median_house_value` del conjunto de entrenamiento `train_set`
- es considerada la variable dependiente o etiqueta en el conjunto de datos
- objetivo `pueda ser utilizado como entrada en el entrenamiento del modelo`.
- acción :
  - usamos DataFrame.drop("${columnName}",axis=${1}) para columnas
    - `df.drop(['B', 'C'], axis=1)`
  - usamos DataFrame.drop("${indexName}",axis=${0}) para index
    - `df.drop([0, 1])`
- visualización del DataFrame traspuesto para que sea mas legible
  - `DataFrame.head().T`


In [4]:
# Procesado de datos
# Guardamos la variable dependiente (etiquetas)
housing_labels = train_set["median_house_value"].copy()
# Eliminamos la columna de la variable dependiente
# Sobrescribimos los datos en `proc_data` Processed data
proc_data = train_set.drop("median_house_value", axis=1)
print("ya no tenemos median_house_value")
# mostramos el código traspuesto para que muestre todas las columnas
print(proc_data.head().T)

ya no tenemos median_house_value
                        7525       6243       17209   19571   3067 
longitude             -118.24    -117.98    -119.72 -120.94 -119.25
latitude                33.91      34.05      34.43   37.59   35.77
housing_median_age       37.0       35.0       33.0    16.0    35.0
total_rooms            1607.0     2342.0     1028.0  3964.0  1618.0
total_bedrooms          377.0      426.0      377.0   824.0   378.0
population             1526.0     2176.0      753.0  2622.0  1449.0
households              375.0      416.0      356.0   766.0   398.0
median_income          1.7158     3.7454     2.3454  2.3152  1.6786
ocean_proximity     <1H OCEAN  <1H OCEAN  <1H OCEAN  INLAND  INLAND


In [5]:
print("Identificación de valores no disponibles")
print("Vemos cuantos Valores nulos hay por columna")
# Identificación de valores no disponibles
# en cada columna suma 1 por cada valor nulo que tenga
null_values_per_column = (
    # el método isnull() es un alias de isna()
    proc_data.isna().sum()
)
print("Column\t\t (count) null val")
print(null_values_per_column)

Identificación de valores no disponibles
Vemos cuantos Valores nulos hay por columna
Column		 (count) null val
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        163
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64


##### identificamos las celdas con valores nulos

`null_rows_idx = proc_data.isnull().any(axis=1)`

- Este código crea una Serie booleana
- indica qué filas del DataFrame proc_data contienen al menos un valor nulo.
- Utiliza el método isnull() para identificar los valores nulos en cada celda del DataFrame,
- y luego any(axis=1) para verificar si al menos uno de los valores a lo largo del eje de las columnas (eje 1) es nulo en cada fila.
- Ouput un Objeto Serie con el Index de la celda y el valor falso


In [6]:
# índices de las filas con valores nulos
null_rows_idx = proc_data.isnull().any(axis=1)

# Imprimir las primeras filas de las filas con valores nulos
print("Filas con valores nulos:\n")
print(proc_data.loc[null_rows_idx].head(20).T)

Filas con valores nulos:

                        10761     18914      7330       7654       7316   \
longitude             -117.87   -122.22    -118.17    -118.26    -118.19   
latitude                33.62     38.15      33.98      33.83      33.98   
housing_median_age        8.0       7.0       41.0       24.0       36.0   
total_rooms            1266.0    5129.0      756.0     3059.0     4179.0   
total_bedrooms            NaN       NaN        NaN        NaN        NaN   
population              375.0    2824.0      873.0     2064.0     4582.0   
households              183.0     738.0      212.0      629.0     1196.0   
median_income           9.802    5.5138     2.7321     3.5518     2.0087   
ocean_proximity     <1H OCEAN  NEAR BAY  <1H OCEAN  <1H OCEAN  <1H OCEAN   

                        4591    6052        16885      20372     19060  \
longitude             -118.28 -117.76      -122.4    -118.88   -122.41   
latitude                34.06   34.04       37.58      34.17     

In [7]:
# Imprimir el tipo de datos de null_rows_idx
print("\nTipo de datos de null_rows_idx:")
print(type(null_rows_idx))
print("Tipo de datos de la serie [", null_rows_idx.dtype, "]")
# Imprimir null_rows_idx para ver sus valores
print("\n[Index] [Value] de null_rows_idx:")
print(null_rows_idx.head())


Tipo de datos de null_rows_idx:
<class 'pandas.core.series.Series'>
Tipo de datos de la serie [ bool ]

[Index] [Value] de null_rows_idx:
7525     False
6243     False
17209    False
19571    False
3067     False
dtype: bool


##### `Listwise deletion`

- [opción 1] Eliminación de las filas con valores nulos
  - 1a. eliminar aquellas instancias incompletas
  - 1b. eliminar toda fila que tenga un valor na en cualquier columna
- [opción 2] Eliminar la columna entera
  - 2a. eliminar la columna por su nombre
  - 2b. eliminar todas las las columnas con valores nulos
  - 2c. modificar los valores nulos de la columna por el valor de la media aritmética


In [8]:
# Eliminación de las filas con valores nulos `Listwise deletion'
# [opción 1] Eliminación de las filas con valores nulos
#        - 1a. eliminar aquellas instancias incompletas,
#             aunque es problemático porque estamos eliminando información
housing_option1 = proc_data.dropna(subset=["total_bedrooms"])
housing_option1.loc[
    null_rows_idx
].head()  # comprobamos que se han eliminado las filas con valores nulos

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


In [9]:
#         - 1b. eliminar toda fila que tenga un valor na en cualquier columna
# eliminamos las filas con valores nulos
housing_option1b = proc_data.dropna(axis=0)
housing_option1b.loc[
    null_rows_idx
].head()  # comprobamos que se han eliminado las filas con valores nulos

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


In [10]:
# [opción 2] Eliminar la columna entera
#    - 2a. eliminar la columna por su nombre

"""
Eliminar la columna entera es una opción si no es una variable importante,
pero en este caso parece que sí lo es dado que, 
aunque esa *feature* no es la que más correlaciona directamente con la variable objetivo,
es una de las dos con las que se calcula `bedrooms_ratio`,
que sí es la segunda más correlacionada.
"""
housing_option2 = proc_data.drop(columns="total_bedrooms")
housing_option2.loc[null_rows_idx].head()
# Si buscamos nulos ahora en housing_option2, no los encontraremos
housing_option2.isnull().any(
    axis=None
)  # comprobamos que no hay valores nulos en el dataset

False

In [11]:
#    - 2b. eliminar todas las las columnas con valores nulos

# También podríamos eliminar directamente todas las columnas con nulos:
proc_data.dropna(axis=1).isnull().any(axis=None)

False

In [12]:
#   - 2c. modificar los valores nulos de la columna por el valor de la media aritmética

# Imputación de algún valor (la mediana en este caso)
median = proc_data["total_bedrooms"].median()
housing_option3 = proc_data["total_bedrooms"].fillna(median)
print(housing_option3.loc[null_rows_idx].head().T)

10761    434.0
18914    434.0
7330     434.0
7654     434.0
7316     434.0
Name: total_bedrooms, dtype: float64


In [13]:
#################################################################
#                     Con  SimpleImputer                        #
#################################################################
# Creamos una instancia de `SimpleImputer`
# indicando que queremos imputar los valores nulos con la mediana,
#  luego usamos el método `fit()` para calcular la mediana de cada columna
# el método `transform()` para aplicar la imputación a todas las columnas.

# Vamos a ver cómo se aplicaría este método a todos los campos numéricos del dataframe
# (recordemos que 'ocean_proximity' es categorial -valores de texto-,
# y no se puede calcular la mediana de un texto).

# También se podría utilizar directamente el método `fit_transform()` de `SimpleImputer`
# para calcular el valor a imputar (con `fit()`)
# y aplicarla (con `transform()`) en un solo paso.
#################################################################
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Carga el conjunto de datos
housing = pd.read_csv("./data/housing.csv")

# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
train_set, test_set = train_test_split(
    housing,
    test_size=0.2,
    stratify=pd.cut(
        housing["median_income"],
        bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
        labels=[1, 2, 3, 4, 5],
    ),
)

# Procesado de datos
housing = train_set.drop(
    "median_house_value", axis=1
)  # Eliminamos la columna de la variable dependiente
housing_labels = train_set[
    "median_house_value"
].copy()  # Guardamos la variable dependiente (etiquetas)
# Mostramos los primeros registros del DataFrame transpuesto
print(housing.head().T)

# Identificación de valores no disponibles
# Índices de las filas con valores nulos
null_rows_idx = housing.isnull().any(axis=1)
print(
    "Número de filas con valores nulos:", null_rows_idx.sum()
)  # Mostramos el número de filas con valores nulos

# Creamos una instancia de `SimpleImputer`
# indicando que queremos imputar los valores nulos con la mediana
imputer = SimpleImputer(strategy="median")

# Seleccionamos las columnas numéricas
housing_num = housing.select_dtypes(include=[np.number])

# Calculamos la mediana de cada columna numérica
imputer.fit(housing_num)

# Reemplazamos los valores nulos por la mediana
housing_tr = pd.DataFrame(
    # trasformamos los datos nulos por los datos de la mediana
    imputer.transform(housing_num),
    columns=housing_num.columns,
    index=housing_num.index,
)

# Mostramos los primeros registros del DataFrame transformado
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print(housing_tr.loc[null_rows_idx].head())

                       15931      10253   3007       15124      8769 
longitude            -122.41    -117.87 -119.01    -116.88    -118.33
latitude               37.73      33.88   35.24      32.86      33.79
housing_median_age      42.0       25.0     6.0        9.0       29.0
total_rooms           2604.0     1808.0    80.0     3049.0     4389.0
total_bedrooms         573.0      440.0    16.0      471.0      873.0
population            1703.0     1342.0    66.0     1527.0     2069.0
households             507.0      454.0    21.0      515.0      901.0
median_income         3.4231      3.025   3.125     5.0733     4.1071
ocean_proximity     NEAR BAY  <1H OCEAN  INLAND  <1H OCEAN  <1H OCEAN
Número de filas con valores nulos: 169
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
3778     -118.40     34.17                24.0       6347.0           431.0   
7654     -118.26   

In [14]:
#################################################################
#       con algoritmo **K-Nearest Neighbors (KNN)**             #
#################################################################
# Existen métodos más avanzados como el uso de **módelos de predicción**
# (tratando la columna con valores nulos como la variable objetivo
# y el resto de columnas como *features*). Por ejemplo,
# podría utilizarse el algoritmo **K-Nearest Neighbors (KNN)**
# para predecir los valores nulos de 'total_bedrooms'
# basándonos en los registros sí etiquetados.
#################################################################
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

# Carga el conjunto de datos
housing = pd.read_csv("./data/housing.csv")

# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
train_set, test_set = train_test_split(
    housing,
    test_size=0.2,
    stratify=pd.cut(
        housing["median_income"],
        bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
        labels=[1, 2, 3, 4, 5],
    ),
)

# Procesado de datos
housing = train_set.drop(
    "median_house_value", axis=1
)  # Eliminamos la columna de la variable dependiente
housing_labels = train_set[
    "median_house_value"
].copy()  # Guardamos la variable dependiente (etiquetas)
# Mostramos los primeros registros del DataFrame transpuesto
print(housing.head().T)

# Identificación de valores no disponibles
# Índices de las filas con valores nulos
null_rows_idx = housing.isnull().any(axis=1)
print(
    "Número de filas con valores nulos:", null_rows_idx.sum()
)  # Mostramos el número de filas con valores nulos

# Creamos una instancia de `KNNImputer`
# para imputar los valores nulos con la estrategia de vecinos más cercanos
imputer = KNNImputer(
    n_neighbors=5
)  # Como argumento ajustas el número de vecinos según tus necesidades

# Seleccionamos las columnas numéricas
housing_num = housing.select_dtypes(include=[np.number])

# Imputamos los valores nulos utilizando KNN
housing_tr = pd.DataFrame(
    # primero el fit ajusta los datos
    # fit : imputador "aprende" de los datos (vecino mas cercanos)
    #       y calcula cualquier estadística necesaria
    # segundo el transform trasforma los datos nulos
    # transform : transforma los datos de entrenamiento nulos en valores
    #              teniendo aprendiendo de los vecinos cercanos
    imputer.fit_transform(housing_num),
    columns=housing_num.columns,
    index=housing_num.index,
)

# Mostramos los primeros registros del DataFrame transformado
print(
    "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -"
)
print(housing_tr.loc[null_rows_idx].head())

                     1240      1714    18714      15125      10982
longitude          -120.19   -122.33 -122.42    -116.86    -117.83
latitude             38.42      38.0   40.57      32.87      33.75
housing_median_age    11.0      35.0    10.0       17.0       22.0
total_rooms         1568.0    3779.0  7949.0     5799.0     6433.0
total_bedrooms       369.0     711.0  1309.0      921.0     1174.0
population            82.0    2493.0  3176.0     2630.0     2703.0
households            33.0     679.0  1163.0      843.0     1125.0
median_income        3.125    2.9781  4.1099     5.0524     4.9957
ocean_proximity     INLAND  NEAR BAY  INLAND  <1H OCEAN  <1H OCEAN
Número de filas con valores nulos: 172
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
       longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
11512    -118.10     33.74                32.0       2035.0           528.6   
20372    -118.88     34.17                15.0   